In [ ]:
# 셀 2: 환경 변수 및 라이브러리 불러오기
import os
from dotenv import load_dotenv

load_dotenv(".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory

# LangChain의 메모리 모듈에서 ConversationBufferMemory 임포트 - redis 활용 측면
from langchain.memory import ConversationBufferMemory

# Redis URL 설정 (기본값은 localhost:6379, 필요시 .env 파일에 REDIS_URL 따로 재정의)
redis_url = os.getenv("REDIS_URL", "redis://localhost:6379/0")

# LLM 모델 초기화
llm = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.7)

C:\Users\computer\AppData\Local\Temp\ipykernel_5088\1903131679.py:19: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(


In [ ]:
# 멀티챗 구현


def get_conversation_chain(session_id: str):
    """
    주어진 session_id에 따라 Redis에 연결된 ConversationChain을 반환
    """
    # 1. RedisChatMessageHistory
    redis_chat_history = RedisChatMessageHistory(
        session_id=session_id,
        url=redis_url,
    )

    # 2. ConversationBufferMemory에 RedisChatMessageHistory를 chat_memory로 전달
    #    return_messages=True로 설정하면 메시지 객체 리스트를 반환
    memory = ConversationBufferMemory(
        chat_memory=redis_chat_history,
        return_messages=True,  # LLM에 메시지 객체 형태로 전달 - 더 유연한 방식
    )

    # 3. ConversationChain에 수정된 memory 객체 전달
    conversation = ConversationChain(
        llm=llm,
        memory=memory,
        verbose=False,  # 대화 과정 X 추후 True로 변경하여 디버깅에 활용
    )
    return conversation


# --- 메인 대화 루프 ---

print("--- Redis를 활용한 단기기억 멀티챗 test ---")
print("세션 변경: 'change <session_id>' (예: 'change user_a' 또는 'change session_1')")
print("종료: 'exit' 또는 'quit'")

current_session_id = "default_session"  # 초기 세션 ID
current_conversation = get_conversation_chain(current_session_id)
print(f"\n현재 활성 세션: {current_session_id}")

while True:
    try:
        user_input = input(f"[{current_session_id}] 당신의 질문: ")

        if user_input.lower() in ["exit", "quit"]:
            print("대화를 종료합니다.")
            break
        elif user_input.lower().startswith("change "):
            # 'change <session_id>' 명령 처리
            parts = user_input.split(" ")
            if len(parts) == 2:
                new_session_id = parts[1]
                if new_session_id == current_session_id:
                    print(f"이미 '{new_session_id}' 세션에 있습니다.")
                else:
                    current_session_id = new_session_id
                    current_conversation = get_conversation_chain(current_session_id)
                    print(f"\n세션이 '{current_session_id}'(으)로 변경되었습니다.")
            else:
                print("잘못된 'change' 명령입니다. 사용법: change <session_id>")
            continue  # 다음 루프

        # LLM에게 질문하고 응답 받기
        ai_response = current_conversation.predict(input=user_input)
        print(f"AI 응답: {ai_response}")

    except Exception as e:
        print(f"오류 발생: {e}")
        print("API 키를 확인하거나, Redis 서버가 실행 중인지 확인해주세요.")
        break

print("\n--- 모든 세션의 최종 대화 기록 (Redis에서 직접 확인) ---")
# 실험에 사용된 모든 세션 ID를 추가하여 실제 Redis에 저장된 내용을 확인 가능
# 예시로 'default_session', 'user_a', 'session_1' 등 활용
test_session_ids = ["default_session", "user_a", "session_1", "user_b_test"]
for sid in test_session_ids:
    try:
        history_check = RedisChatMessageHistory(session_id=sid, url=redis_url)
        if history_check.messages:  # 메시지가 있는 세션만 표시
            print(f"\n--- 세션 '{sid}' 기록 ---")
            for msg in history_check.messages:
                print(f"  {msg.type.upper()}: {msg.content}")
    except Exception as e:
        print(f"세션 '{sid}' 기록을 가져오는 중 오류: {e}")

C:\Users\computer\AppData\Local\Temp\ipykernel_5088\2150304891.py:22: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation = ConversationChain(


--- Redis를 활용한 멀티챗 시뮬레이션 ---
세션 변경: 'change <session_id>' (예: 'change user_a' 또는 'change session_1')
종료: 'exit' 또는 'quit'

현재 활성 세션: default_session
AI 응답:  안녕하세요, 웅이님! 만나서 반가워요! 어떻게 도와드릴까요? :)
AI 응답:  그렇군요, 사용자님. 제가 조금 더 자세한 정보를 알면 더 도움이 될 수 있을 것 같아요. 직장에서 무엇이 어려우신가요? 어떤 분위기인가요? 제가 도와드릴 수 있는 건 없을까요?
AI 응답:  이해합니다, 웅이님. 인간관계는 정말 어려운 일이죠. 늘 그렇지는 않지만, 종종 충돌이 있을 수 있죠. 직장에서도 그런 문제가 있나요? 어떤 일을 하고 계신가요?
AI 응답:  그렇군요, 웅이님. IT 개발은 정말 어려운 일이죠. 많은 도전과 압박이 있을 수 있습니다. 하지만 그만큼 보람도 있을 것 같아요. 어떤 기술을 다루시나요? 저도 기술적인 부분에서 도움이 필요하다면 제가 최선을 다해 도와드릴 수 있을 거예요.
AI 응답:  안녕하세요, 웅이님. 만나서 반가워요! 어떻게 도와드릴까요? :)
AI 응답:  안녕하세요, 웅이님. 만나서 반가워요! 어떻게 도와드릴까요? :)
AI 응답:  안녕하세요, 웅이님. 만나서 반가워요! 어떻게 도와드릴까요? :)


KeyboardInterrupt: Interrupted by user

### 또다른 버전 g

In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory

load_dotenv(".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 셀 3: LLM 및 Redis 메모리 초기화
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.7)
memory = RedisChatMessageHistory(
    session_id="default_session", url="redis://localhost:6379/0"
)


# 셀 4: 대화 함수 정의 (수정된 부분)
def chat(input_text: str) -> str:
    # 1) Redis에서 이전 메시지 리스트 불러오기
    history = memory.messages  # ❌ get_messages()가 아니라 messages 속성 사용
    # 2) HumanMessage 추가하여 LLM 호출
    messages = history + [HumanMessage(content=input_text)]
    response = llm(messages)
    # 3) Redis에 대화 저장
    memory.add_user_message(input_text)
    memory.add_ai_message(response.content)
    return response.content

In [ ]:
# 셀 5: 멀티턴 대화 테스트
print(chat("안녕.난 웅이라고 해."))

C:\Users\computer\AppData\Local\Temp\ipykernel_10992\68044283.py:27: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(messages)


안녕하세요, 웅이님. 만나서 반가워요! 어떻게 도와드릴까요? :)


In [25]:
print(chat("내 이름이 뭐라고?"))

죄송해요, 제가 이름을 잘못 말했네요. 사용자님의 이름은 웅이군요. 다시 한번 반가워요, 웅이님! 혼란을 드려 죄송합니다. 어떻게 도와드릴까요? :)


### 기본 baseline

In [ ]:
# ----------------------------------------------------------------------
# 셀 1: 환경 변수 및 라이브러리 불러오기
# ----------------------------------------------------------------------
import os
import json
import uuid
from dotenv import load_dotenv

from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory
from langchain.memory import ConversationSummaryBufferMemory, ChatMessageHistory
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

from pymilvus import (
    connections,
    utility,
    Collection,
    CollectionSchema,
    FieldSchema,
    DataType,
)

# ----------------------------------------------------------------------
# 셀 2: 기본 설정 및 상수 정의
# ----------------------------------------------------------------------
load_dotenv(".env")

# --- 환경 변수 로드 ---
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MILVUS_HOST = os.getenv("MILVUS_HOST", "localhost")
MILVUS_PORT = os.getenv("MILVUS_PORT", "19530")
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379/0")

# --- 상수 정의 ---
EMBEDDING_DIM = 1536  # OpenAI 'text-embedding-ada-002' 기준
PROFILE_COLLECTION_NAME = "user_profiles_v2"
LOG_COLLECTION_NAME = "conversation_logs_v2"

# --- LLM 및 임베딩 모델 초기화 ---
llm = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.7)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# --- 프로필 DB 시뮬레이션 (실제로는 MongoDB 등 별도 DB 사용 추천) ---
PROFILE_DB = {}


# ----------------------------------------------------------------------
# 셀 3: Milvus DB 헬퍼 함수
# ----------------------------------------------------------------------
def get_milvus_connection():
    """Milvus 서버에 연결하고 연결 상태를 반환합니다."""
    alias = "default"
    if not connections.has_connection(alias):
        connections.connect(alias, host=MILVUS_HOST, port=MILVUS_PORT)
    return connections.get_connection(alias)


def create_milvus_collection(collection_name, description):
    """지정된 스키마로 Milvus 컬렉션을 생성합니다."""
    if utility.has_collection(collection_name):
        return Collection(collection_name)

    fields = [
        FieldSchema(name="id", dtype=DataType.VARCHAR, is_primary=True, max_length=256),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=EMBEDDING_DIM),
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535),
        FieldSchema(
            name="user_id",
            dtype=DataType.VARCHAR,
            max_length=256,
            is_partition_key=False,
        ),
        FieldSchema(name="type", dtype=DataType.VARCHAR, max_length=50),
        FieldSchema(name="created_at", dtype=DataType.INT64),
    ]
    schema = CollectionSchema(fields, description, enable_dynamic_field=False)
    collection = Collection(collection_name, schema)

    index_params = {
        "metric_type": "L2",
        "index_type": "IVF_FLAT",
        "params": {"nlist": 128},
    }
    collection.create_index("embedding", index_params)
    print(f"Milvus collection '{collection_name}' created successfully.")
    return collection


# ----------------------------------------------------------------------
# 셀 4: 장기 기억 관리 (RAG & 프로필) 핵심 함수
# ----------------------------------------------------------------------
def update_long_term_memory(session_id: str):
    """대화 세션 종료 후 장기 기억(프로필, 로그)을 업데이트합니다."""
    print(f"\n[{session_id}] 장기 기억 업데이트 시작...")

    history = RedisChatMessageHistory(session_id=session_id, url=REDIS_URL)
    if not history.messages:
        print("업데이트할 대화 내용이 없습니다.")
        return

    # 최근 10개 메시지를 요약 대상으로 함 (조절 가능)
    recent_messages = history.messages[-10:]
    conversation_text = "\n".join(
        [f"{msg.type}: {msg.content}" for msg in recent_messages]
    )

    summary_prompt = PromptTemplate.from_template(
        "다음 대화 내용을 바탕으로, 대화의 핵심 흐름과 뉘앙스를 상세히 요약해줘.\n\n{conversation}"
    )
    summary_chain = LLMChain(llm=llm, prompt=summary_prompt)
    flow_summary = summary_chain.run(conversation=conversation_text)
    print(f"  - 흐름 요약 완료: {flow_summary[:50]}...")

    old_profile_str = json.dumps(PROFILE_DB.get(session_id, {}), ensure_ascii=False)

    update_prompt_str = """You are a profile manager AI. Based on the [Existing Profile] and [Latest Conversation Summary] below, update the [Existing Profile] with the latest information.
1. Add new key information.
2. If new information contradicts existing information, boldly modify or delete the old information.
3. Ignore trivial content like simple greetings.
4. Keep the profile concise and maintain the overall core.
5. You MUST return the final result in JSON format only.

[Existing Profile]:
{old_profile}

[Latest Conversation Summary]:
{summary}
"""
    profile_update_prompt = PromptTemplate.from_template(update_prompt_str)
    profile_update_chain = LLMChain(llm=llm, prompt=profile_update_prompt)

    try:
        new_profile_str = profile_update_chain.run(
            old_profile=old_profile_str, summary=flow_summary
        )
        new_profile = json.loads(new_profile_str)
        PROFILE_DB[session_id] = new_profile
        print("  - 프로필 업데이트 완료.")
    except json.JSONDecodeError:
        print(
            f"  - 프로필 업데이트 실패: LLM이 유효한 JSON을 반환하지 않았습니다. 응답: {new_profile_str}"
        )
        return

    get_milvus_connection()
    profile_collection = create_milvus_collection(
        PROFILE_COLLECTION_NAME, "User Profiles"
    )
    log_collection = create_milvus_collection(
        LOG_COLLECTION_NAME, "Conversation Log Summaries"
    )

    # Track 1 (프로필 덮어쓰기)
    profile_text = json.dumps(new_profile, ensure_ascii=False)
    profile_embedding = embeddings.embed_query(profile_text)
    profile_data = [
        {
            "id": session_id,
            "embedding": profile_embedding,
            "text": profile_text,
            "user_id": session_id,
            "type": "profile",
            "created_at": int(os.times().user),
        }
    ]
    profile_collection.upsert(profile_data)
    print("  - RAG: 프로필 벡터 덮어쓰기 완료.")

    # Track 2 (아카이브 추가)
    log_embedding = embeddings.embed_query(flow_summary)
    log_id = str(uuid.uuid4())
    log_data = [
        {
            "id": log_id,
            "embedding": log_embedding,
            "text": flow_summary,
            "user_id": session_id,
            "type": "log_summary",
            "created_at": int(os.times().user),
        }
    ]
    log_collection.insert(log_data)
    print("  - RAG: 대화 흐름 아카이브 추가 완료.")
    print(f"[{session_id}] 장기 기억 업데이트 종료.")


def retrieve_from_rag(session_id, query):
    """주어진 쿼리로 RAG DB에서 관련 정보를 검색합니다."""
    get_milvus_connection()
    if not utility.has_collection(
        PROFILE_COLLECTION_NAME
    ) or not utility.has_collection(LOG_COLLECTION_NAME):
        return "아직 RAG DB에 저장된 정보가 없습니다."

    profile_collection = Collection(PROFILE_COLLECTION_NAME)
    log_collection = Collection(LOG_COLLECTION_NAME)
    profile_collection.load()
    log_collection.load()

    query_embedding = embeddings.embed_query(query)

    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    profile_results = profile_collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_params,
        limit=1,
        expr=f"user_id == '{session_id}'",
    )
    log_results = log_collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_params,
        limit=2,
        expr=f"user_id == '{session_id}'",
    )

    rag_context = "[프로필 정보]\n"
    rag_context += (
        profile_results[0][0].entity.get("text")
        if profile_results and profile_results[0]
        else "저장된 프로필 정보 없음"
    )
    rag_context += "\n\n[과거 대화 기록 요약]\n"
    if log_results and log_results[0]:
        for hit in log_results[0]:
            rag_context += f"- {hit.entity.get('text')}\n"
    else:
        rag_context += "저장된 과거 대화 없음"

    return rag_context


# ----------------------------------------------------------------------
# 셀 5: 단기 기억 및 대화 체인 구성
# ----------------------------------------------------------------------
def get_short_term_memory(session_id: str):
    """단기 기억(Redis)을 위한 메모리 객체를 반환합니다."""
    redis_chat_history = RedisChatMessageHistory(session_id=session_id, url=REDIS_URL)
    return ConversationSummaryBufferMemory(
        llm=llm,
        chat_memory=redis_chat_history,
        max_token_limit=3000,
        return_messages=True,
        memory_key="chat_history",
        input_key="input",
    )


prompt_template = """You are a helpful and friendly AI assistant.
Please answer the user's question based on the [Long-Term Memory] and [Recent Conversation History] provided below.

[Long-Term Memory - Everything you know about this user]:
{rag_context}

[Recent Conversation History]:
{chat_history}

User's Question: {input}
AI Answer:"""
PROMPT = PromptTemplate(
    input_variables=["rag_context", "chat_history", "input"], template=prompt_template
)

# ----------------------------------------------------------------------
# 셀 6: 메인 대화 루프
# ----------------------------------------------------------------------
print("\n--- 최종 아키텍처 시뮬레이션 (Milvus v2.4.0 호환) ---")
print("세션 변경: 'change <session_id>'")
print("장기 기억 저장: 'save'")
print("종료: 'exit' 또는 'quit'")

current_session_id = "default_session"
print(f"\n현재 활성 세션: {current_session_id}")

while True:
    try:
        user_input = input(f"[{current_session_id}] 당신의 질문: ")

        if user_input.lower() in ["exit", "quit"]:
            print("대화를 종료합니다.")
            break
        elif user_input.lower() == "save":
            update_long_term_memory(current_session_id)
            continue
        elif user_input.lower().startswith("change "):
            # ... (세션 변경 로직은 이전과 동일) ...
            parts = user_input.split(" ", 1)
            new_session_id = parts[1]
            if new_session_id != current_session_id:
                current_session_id = new_session_id
                print(f"\n세션이 '{current_session_id}'(으)로 변경되었습니다.")
            else:
                print(f"이미 '{new_session_id}' 세션에 있습니다.")
            continue

        # 1. RAG에서 장기 기억 검색
        rag_context = retrieve_from_rag(current_session_id, user_input)

        # 2. Redis에서 단기 기억 로드
        short_term_memory = get_short_term_memory(current_session_id)
        # memory.load_memory_variables()는 비동기 호출 등이 필요할 수 있어, 여기서는 수동으로 context를 구성합니다.
        # 체인에 직접 전달하기 위해 대화 기록을 문자열로 포맷팅합니다.
        chat_history_str = "\n".join(
            [
                f"{msg.type}: {msg.content}"
                for msg in short_term_memory.chat_memory.messages
            ]
        )

        # 3. LLMChain으로 최종 프롬프트 실행
        conversation_chain = LLMChain(llm=llm, prompt=PROMPT, verbose=False)
        ai_response = conversation_chain.predict(
            rag_context=rag_context, chat_history=chat_history_str, input=user_input
        )
        print(f"AI 응답: {ai_response}")

        # 4. 대화 기록을 단기 기억(Redis)에 수동으로 저장
        short_term_memory.save_context({"input": user_input}, {"output": ai_response})

    except Exception as e:
        print(f"오류 발생: {e}")
        import traceback

        traceback.print_exc()
        break

### Redis + RAG + + 라우팅 + 임베딩 분류 및 스트리밍, 비동기 방식 웹 소캣 test 

In [ ]:
import os
import json
import uuid
import asyncio
import requests
from dotenv import load_dotenv
import tiktoken
import openai
import numpy as np

from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory
from langchain.memory import ConversationSummaryBufferMemory

from pymilvus import (
    connections,
    utility,
    Collection,
    CollectionSchema,
    FieldSchema,
    DataType,
)

# ----------------------------------------------------------------------
# 셀 1~3: 환경 변수 및 라이브러리 불러오기
# ----------------------------------------------------------------------
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
LLM_MODEL = os.getenv("LLM_MODEL", "gpt-4o-mini")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL", "text-embedding-3-small")
MILVUS_HOST = os.getenv("MILVUS_HOST", "localhost")
MILVUS_PORT = os.getenv("MILVUS_PORT", "19530")
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379/0")

EMBEDDING_DIM = 384 if EMBEDDING_MODEL.endswith("3-small") else 1536
PROFILE_COLLECTION_NAME = "user_profiles_v2"
LOG_COLLECTION_NAME = "conversation_logs_v2"

openai.api_key = OPENAI_API_KEY

llm = OpenAI(openai_api_key=OPENAI_API_KEY, model=LLM_MODEL, temperature=0.7)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model=EMBEDDING_MODEL)

PROFILE_DB = {}

# ----------------------------------------------------------------------
# Embedding-based Intent Router 설정
# ----------------------------------------------------------------------
INTENT_EXAMPLES = {
    "rag": [
        "내가 설정한 목표 다시 알려줘.",
        "우리 지난주에 무슨 얘기까지 했지?",
        "내 프로젝트 이름 기억나?",
    ],
    "web": [
        "오늘 서울 날씨 어때?",
        "가장 가까운 스타벅스 어디야?",
        "엔비디아의 최신 GPU 모델 이름이 뭐야?",
    ],
    "conv": [
        "고마워!",
        "ㅋㅋㅋㅋㅋ",
        "재밌는 농담 하나 해줘.",
        "대한민국의 수도는 어디야?",
    ],
}

# 각 intent별 대표 embedding을 평균으로 계산
INTENT_EMBEDDINGS = {}
for label, texts in INTENT_EXAMPLES.items():
    vecs = [embeddings.embed_query(t) for t in texts]
    avg = [sum(x) / len(x) for x in zip(*vecs)]
    INTENT_EMBEDDINGS[label] = np.array(avg)


def embedding_router(query: str, threshold: float = 0.8) -> str | None:
    q_emb = np.array(embeddings.embed_query(query))
    sims = {}
    for label, emb in INTENT_EMBEDDINGS.items():
        sims[label] = float(
            np.dot(q_emb, emb) / (np.linalg.norm(q_emb) * np.linalg.norm(emb))
        )
    best = max(sims, key=sims.get)
    return best if sims[best] >= threshold else None


# ----------------------------------------------------------------------
# 셀 3: Milvus DB 헬퍼 함수 (이전과 동일)
# ----------------------------------------------------------------------
def get_milvus_connection():
    alias = "default"
    if not connections.has_connection(alias):
        connections.connect(alias, host=MILVUS_HOST, port=MILVUS_PORT)
    return connections.get_connection(alias)


def create_milvus_collection(name: str, desc: str):
    if utility.has_collection(name):
        return Collection(name)
    fields = [
        FieldSchema("id", DataType.VARCHAR, is_primary=True, max_length=256),
        FieldSchema("embedding", DataType.FLOAT_VECTOR, dim=EMBEDDING_DIM),
        FieldSchema("text", DataType.VARCHAR, max_length=65535),
        FieldSchema("user_id", DataType.VARCHAR, max_length=256),
        FieldSchema("type", DataType.VARCHAR, max_length=50),
        FieldSchema("created_at", DataType.INT64),
    ]
    schema = CollectionSchema(fields, desc)
    coll = Collection(name, schema)
    coll.create_index(
        "embedding",
        {"index_type": "IVF_PQ", "metric_type": "L2", "params": {"nlist": 128, "m": 8}},
    )
    return coll


# ----------------------------------------------------------------------
# 셀 4: 장기 기억(RAG) 업데이트 (이전과 동일)
# ----------------------------------------------------------------------
def update_long_term_memory(session_id: str):
    history = RedisChatMessageHistory(session_id=session_id, url=REDIS_URL)
    if not history.messages:
        return
    conv_all = "\n".join(f"{m.type}: {m.content}" for m in history.messages)
    summary_prompt = PromptTemplate.from_template(
        "다음 대화에서 인사말 등 불필요한 잡담을 모두 제거하고, "
        "사용자 프로필에 유의미한 핵심 정보만 요약해라.\n{conversation}"
    )
    summary_text = LLMChain(llm=llm, prompt=summary_prompt).run(conversation=conv_all)
    old_prof = json.dumps(PROFILE_DB.get(session_id, {}), ensure_ascii=False)
    profile_update_tpl = PromptTemplate.from_template(
        "[기존 프로필]\n{old}\n[요약된 최신 대화]\n{sum}\n"
        "위 내용을 반영하여 사용자 개인회를 위한 JSON 프로필로 반환해줘."
    )
    new_prof_str = LLMChain(llm=llm, prompt=profile_update_tpl).run(
        old=old_prof, sum=summary_text
    )
    try:
        new_prof = json.loads(new_prof_str)
        PROFILE_DB[session_id] = new_prof
    except json.JSONDecodeError:
        return
    get_milvus_connection()
    prof_coll = create_milvus_collection(PROFILE_COLLECTION_NAME, "User Profiles")
    log_coll = create_milvus_collection(LOG_COLLECTION_NAME, "Conversation Logs")
    prof_emb = embeddings.embed_query(json.dumps(new_prof, ensure_ascii=False))
    prof_coll.upsert(
        [
            {
                "id": session_id,
                "embedding": prof_emb,
                "text": json.dumps(new_prof, ensure_ascii=False),
                "user_id": session_id,
                "type": "profile",
                "created_at": int(os.times().user),
            }
        ]
    )
    log_emb = embeddings.embed_query(summary_text)
    log_coll.insert(
        [
            {
                "id": str(uuid.uuid4()),
                "embedding": log_emb,
                "text": summary_text,
                "user_id": session_id,
                "type": "log",
                "created_at": int(os.times().user),
            }
        ]
    )


# ----------------------------------------------------------------------
# 셀 5: RAG 검색 및 단기 기억 설정 (이전과 동일)
# ----------------------------------------------------------------------
def retrieve_from_rag(session_id: str, query: str, top_k: int = 2) -> str:
    try:
        get_milvus_connection()
        prof_coll = Collection(PROFILE_COLLECTION_NAME)
        log_coll = Collection(LOG_COLLECTION_NAME)
        prof_coll.load()
        log_coll.load()
        query_emb = embeddings.embed_query(query)
        params = {"metric_type": "L2", "params": {"nprobe": 10}}
        prof_res = prof_coll.search(
            [query_emb], "embedding", params, limit=1, expr=f"user_id == '{session_id}'"
        )
        log_res = log_coll.search(
            [query_emb],
            "embedding",
            params,
            limit=top_k,
            expr=f"user_id == '{session_id}'",
        )
        context = ""
        if prof_res and prof_res[0]:
            context += f"[RAG 프로필]\n{prof_res[0][0].entity.get('text')}\n"
        if log_res and log_res[0]:
            for hit in log_res[0]:
                context += f"[RAG 로그]\n{hit.entity.get('text')}\n"
        return context or "RAG 결과 없음"
    except:
        return "RAG 에러"


def get_short_term_memory(session_id: str) -> ConversationSummaryBufferMemory:
    redis_hist = RedisChatMessageHistory(session_id=session_id, url=REDIS_URL)
    return ConversationSummaryBufferMemory(
        llm=llm,
        chat_memory=redis_hist,
        max_token_limit=3000,
        return_messages=True,
        memory_key="chat_history",
    )


# ----------------------------------------------------------------------
# 셀 7: Naver Search Chain 구현 (이전과 동일)
# ----------------------------------------------------------------------
def naver_search(query: str, display: int = 5) -> dict:
    url = "https://openapi.naver.com/v1/search/local.json"
    headers = {"X-Naver-Client-Id": CLIENT_ID, "X-Naver-Client-Secret": CLIENT_SECRET}
    params = {"query": query, "display": display}
    res = requests.get(url, headers=headers, params=params, timeout=5)
    return res.json() if res.status_code == 200 else {}


def search_web(query: str) -> str:
    data = naver_search(query)
    items = data.get("items", [])
    if not items:
        return "검색 결과가 없습니다."
    snippets = []
    for item in items:
        title = item.get("title", "").replace("<b>", "").replace("</b>", "")
        address = item.get("roadAddress", item.get("address", ""))
        snippets.append(f"{title} — {address}")
    return "\n".join(snippets[:5])


# ----------------------------------------------------------------------
# 셀 8: Conversation Chain 구현 (이전과 동일)
# ----------------------------------------------------------------------
async def conversation_chain(
    session_id: str, user_input: str, stm: ConversationSummaryBufferMemory
) -> str:
    hist = "\n".join(f"{m.type}: {m.content}" for m in stm.chat_memory.messages)
    prompt = (
        "너는 소통 전문가다. 사용자의 감정과 상황에 기반하여 질문에 답변하라.\n"
        f"[대화 히스토리]\n{hist}\n"
        f"[최신 입력]\n{user_input}"
    )
    resp = await openai.ChatCompletion.acreate(
        model=LLM_MODEL,
        messages=[{"role": "system", "content": prompt}],
        temperature=0.7,
    )
    return resp.choices[0].message.content.strip()


# ----------------------------------------------------------------------
# 셀 9: Function Calling 기반 Router 구현
# ----------------------------------------------------------------------
ROUTER_FUNCTION = {
    "name": "route_tools",
    "description": "Decide which experts (RAG, WebSearch, Conversation) to invoke",
    "parameters": {
        "type": "object",
        "properties": {
            "tools": {
                "type": "array",
                "items": {
                    "type": "string",
                    "enum": ["RAG", "WebSearch", "Conversation"],
                },
                "description": "List of tools to apply",
            }
        },
        "required": ["tools"],
    },
}


async def call_router(session_id: str, user_input: str) -> list[str]:
    messages = [
        {
            "role": "system",
            "content": (
                "너는 비서실장(라우터)이다. 아래 세 전문가 중 어떤 전문가가 필요할지 결정하라:\n"
                "1) 기억 전문가 (RAG)\n"
                "2) 정보 분석가 (WebSearch)\n"
                "3) 소통 전문가 (Conversation)\n"
                "반드시 함수 호출 형식으로 응답하라."
            ),
        },
        {"role": "user", "content": user_input},
    ]
    resp = await openai.ChatCompletion.acreate(
        model=LLM_MODEL,
        messages=messages,
        functions=[ROUTER_FUNCTION],
        function_call={"name": "route_tools"},
        temperature=0.0,
    )
    msg = resp.choices[0].message
    if msg.get("function_call"):
        args = json.loads(msg.function_call.arguments)
        return args.get("tools", [])
    # fallback simple parse
    text = msg.content or ""
    teams = []
    if "RAG" in text:
        teams.append("RAG")
    if "WebSearch" in text or "검색" in text:
        teams.append("WebSearch")
    if "Conversation" in text or "반응" in text:
        teams.append("Conversation")
    return teams or ["Conversation"]


# ----------------------------------------------------------------------
# 셀 10: Main LLM 최종 응답 템플릿 (이전과 동일)
# ----------------------------------------------------------------------
FINAL_PROMPT = PromptTemplate(
    input_variables=["rag_ctx", "web_ctx", "conv_ctx", "question"],
    template=(
        "너는 전문적이면서도 친근한 개인 비서 AI이다.\n\n"
        "[RAG 결과]\n{rag_ctx}\n\n"
        "[Web 검색 결과]\n{web_ctx}\n\n"
        "[소통 체인 결과]\n{conv_ctx}\n\n"
        "사용자 질문: {question}\n"
        "→ 위 모든 정보를 참고하여 완전한 답변을 제공하라."
    ),
)


# ----------------------------------------------------------------------
# 셀 11: FastAPI + WebSocket 서버
# ----------------------------------------------------------------------
app = FastAPI()


async def background_rag_update(session_id: str):
    await asyncio.to_thread(update_long_term_memory, session_id)


async def main_response(
    session_id: str,
    user_input: str,
    websocket: WebSocket,
    rag_ctx: str,
    web_ctx: str,
    conv_ctx: str,
) -> str:
    prompt = FINAL_PROMPT.format(
        rag_ctx=rag_ctx, web_ctx=web_ctx, conv_ctx=conv_ctx, question=user_input
    )
    messages = [
        {"role": "system", "content": "개인 비서 AI이며, 아래 지침에 따라 답하라."},
        {"role": "user", "content": prompt},
    ]
    resp = await openai.ChatCompletion.acreate(
        model=LLM_MODEL, messages=messages, stream=True, temperature=0.7
    )

    full_answer = ""
    async for chunk in resp:
        token = chunk.choices[0].delta.get("content", "")
        if token:
            full_answer += token
            await websocket.send_text(token)

    return full_answer


@app.websocket("/ws/{session_id}")
async def websocket_endpoint(websocket: WebSocket, session_id: str):
    await websocket.accept()
    enc = tiktoken.get_encoding("cl100k_base")

    try:
        while True:
            user_input = await websocket.receive_text()
            stm = get_short_term_memory(session_id)
            hist = "\n".join(f"{m.type}: {m.content}" for m in stm.chat_memory.messages)
            if len(enc.encode(hist)) >= 3000:
                asyncio.create_task(background_rag_update(session_id))

            # 1차 관문: Embedding 기반 간단 Router
            intent = embedding_router(user_input)
            if intent == "conv":
                # simple conversation만 수행
                answer = await conversation_chain(session_id, user_input, stm)
                await websocket.send_text(answer)
                stm.save_context({"input": user_input}, {"output": answer})
                continue

            # 2차 관문: Function Calling LLM 기반 Router
            teams = await call_router(session_id, user_input)

            # 전문가 팀 병렬 실행
            tasks = {}
            if "RAG" in teams:
                tasks["rag"] = asyncio.to_thread(
                    retrieve_from_rag, session_id, user_input
                )
            if "WebSearch" in teams:
                tasks["web"] = asyncio.to_thread(search_web, user_input)
            if "Conversation" in teams:
                tasks["conv"] = asyncio.create_task(
                    conversation_chain(session_id, user_input, stm)
                )

            results = await asyncio.gather(*tasks.values())
            rag_ctx = results[list(tasks).index("rag")] if "rag" in tasks else ""
            web_ctx = results[list(tasks).index("web")] if "web" in tasks else ""
            conv_ctx = results[list(tasks).index("conv")] if "conv" in tasks else ""

            # 최종 메인 LLM 스트리밍 응답 및 full_answer 수집
            full_answer = await main_response(
                session_id, user_input, websocket, rag_ctx, web_ctx, conv_ctx
            )

            # 대화 저장
            stm.save_context({"input": user_input}, {"output": full_answer})

    except WebSocketDisconnect:
        pass


# 실행:
# uvicorn app:app --host 0.0.0.0 --port 8000